In [0]:
import sys
sys.path.append("/Workspace/SPTrans_v2.0")

from datetime import datetime
from pyspark.sql.functions import trim, upper, col, length
from core.config_loader import load_config
from core.sptrans_client import SPTransClient

config = load_config("/Workspace/SPTrans_v2.0/config.json")

sp = SPTransClient(
    base_url=config["sptrans"]["base_url"],
    token=config["sptrans"]["token"],
    proxies=config["sptrans"].get("proxy")
)

def coletar_linhas():
    if not sp.authenticate():
        print("❌ Falha na autenticação")
        return

    posicoes_df = spark.table("workspace.sptrans.bronze_posicao")

    # ✅ buscar letreiros distintos
    letreiros_rows = (
        posicoes_df
        .select(trim(upper(col("letreiro"))).alias("letreiro"))
        .where((col("letreiro").isNotNull()) & (length(col("letreiro")) > 0))
        .distinct()
        .collect()
    )

    letreiros = sorted({r.letreiro for r in letreiros_rows})
    print(f"✅ {len(letreiros)} letreiros distintos encontrados")

    if len(letreiros) == 0:
        print("⚠ Nenhum letreiro encontrado. Rode coletar_posicoes() antes.")
        return

    registros = []
    ts_now = datetime.utcnow()

    for i, letra in enumerate(letreiros, start=1):
        print(f"[{i}/{len(letreiros)}] Buscando dados da linha: {letra}")

        resp = sp.get(f"Linha/Buscar?termosBusca={letra}")
        if not resp or not isinstance(resp, list):
            continue

        for item in resp:
            registros.append({
                "fetch_time": ts_now,
                "letreiro": item.get("lt"),
                "tipo": item.get("tp"),
                "letreiro_decimal": item.get("tl"),
                "sentido": item.get("sl"),
                "modo_operacao": item.get("lc"),
                "cod_linha": item.get("cl")
            })

    if not registros:
        print("⚠ Nenhuma informação de linha retornada pela API")
        return

    df = spark.createDataFrame(registros)

    df.write.format("delta") \
        .option("mergeSchema", "true") \
        .mode("overwrite") \
        .saveAsTable("workspace.sptrans.bronze_linhas")

    print(f"✅ {len(registros)} linhas salvas em bronze_linhas")
    display(df.limit(10))

coletar_linhas()


✅ Autenticado na API SPTrans
✅ 904 letreiros distintos encontrados
[1/904] Buscando dados da linha: 1012-10
[2/904] Buscando dados da linha: 1015-10
[3/904] Buscando dados da linha: 1016-10
[4/904] Buscando dados da linha: 1017-10
[5/904] Buscando dados da linha: 1018-10
[6/904] Buscando dados da linha: 1019-10
[7/904] Buscando dados da linha: 1020-10
[8/904] Buscando dados da linha: 1023-10
[9/904] Buscando dados da linha: 1024-10
[10/904] Buscando dados da linha: 1025-10
[11/904] Buscando dados da linha: 1026-10
[12/904] Buscando dados da linha: 1034-10
[13/904] Buscando dados da linha: 1036-10
[14/904] Buscando dados da linha: 106A-10
[15/904] Buscando dados da linha: 107T-10
[16/904] Buscando dados da linha: 1156-10
[17/904] Buscando dados da linha: 1177-10
[18/904] Buscando dados da linha: 1178-10
[19/904] Buscando dados da linha: 118C-10
[20/904] Buscando dados da linha: 118Y-10
[21/904] Buscando dados da linha: 119C-10
[22/904] Buscando dados da linha: 1206-10
[23/904] Buscando 

cod_linha,fetch_time,letreiro,letreiro_decimal,modo_operacao,sentido,tipo
35254,2025-11-09T23:44:26.349Z,1012,10,true,2,JD. MONTE BELO
2486,2025-11-09T23:44:26.349Z,1012,10,true,1,JD. MONTE BELO
2230,2025-11-09T23:44:26.349Z,1015,10,true,1,CHÁC. MARIA TRINDADE
34998,2025-11-09T23:44:26.349Z,1015,10,true,2,CHÁC. MARIA TRINDADE
831,2025-11-09T23:44:26.349Z,1016,10,false,1,SHOP. CENTER NORTE
33599,2025-11-09T23:44:26.349Z,1016,10,false,2,SHOP. CENTER NORTE
2141,2025-11-09T23:44:26.349Z,1017,10,false,1,CONEXÃO VL. IÓRIO
34909,2025-11-09T23:44:26.349Z,1017,10,false,2,CONEXÃO VL. IÓRIO
832,2025-11-09T23:44:26.349Z,1018,10,false,1,METRÔ SANTANA
33600,2025-11-09T23:44:26.349Z,1018,10,false,2,METRÔ SANTANA


In [0]:
%sql
SELECT COUNT(*) AS total_registros
FROM workspace.sptrans.bronze_linhas;

total_registros
3136
